In [1]:
import pandas as pd
import numpy as np

import os
import time
import gc
import json
import requests

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
import io
from pandas.io.json import json_normalize

In [11]:
response = requests.get('https://resultsdb-api.rotogrinders.com/api/contests?start=10/17/2019&end=10/24/2019&lean=true')
json_data = json.loads(response.json)

AttributeError: module 'requests' has no attribute 'json'

In [14]:
json_data = response.json()

In [65]:
json_data

[{'_id': '5da9d829f3fffe417566fbcb',
  'rgPrizePool': 16526,
  'rgPrizeWinnerCount': 1184,
  'entryCount': 13406,
  'prizePool': 50000,
  'slateType': 1,
  'gameCount': 9,
  'siteSlateId': '39493',
  'start': '2019-10-17T23:00:00.000Z',
  'sport': 4,
  'maxEntriesPerUser': None,
  'entryFee': 4.44,
  'name': '$50K Thu NHL Breakaway ($10K to 1st)',
  'siteContestId': '230245439',
  '_slateId': '5da9d828f3fffe417566f966',
  'complete': True,
  'createdAt': '2019-11-16T20:29:33.198Z',
  'playerOwnership': [],
  'prizes': [{'cash': 10000, 'minFinish': 1, 'maxFinish': 1},
   {'cash': 4000, 'minFinish': 2, 'maxFinish': 2},
   {'cash': 2000, 'minFinish': 3, 'maxFinish': 3},
   {'cash': 1000, 'minFinish': 4, 'maxFinish': 4},
   {'cash': 400, 'minFinish': 5, 'maxFinish': 5},
   {'cash': 200, 'minFinish': 7, 'maxFinish': 7},
   {'cash': 150, 'minFinish': 8, 'maxFinish': 8},
   {'cash': 100, 'minFinish': 9, 'maxFinish': 9},
   {'cash': 75, 'minFinish': 12, 'maxFinish': 14},
   {'cash': 58.33, 'mi

In [16]:
df = pd.DataFrame(list(json_data['prizes']))

TypeError: list indices must be integers or slices, not str

In [105]:
df = pd.DataFrame(json_data, columns = ['name',
                                        'gameCount',
                                        'start',
                                        'sport',
                                        'prizePool',
                                        'entryFee',
                                        'maxEntriesPerUser',
                                        'maxEntries',
                                        'entryCount',
                                        ])

In [167]:
nfl = rg_df(json_data, 'nfl')
nhl = rg_df(json_data, 'nhl')

In [171]:
pd.options.display.float_format = "{:.2f}".format

In [172]:
nfl.describe()

,gameCount,prizePool,entryFee,maxEntriesPerUser,maxEntries,entryCount
count,80.00,80.00,80.00,80.00,80.00,80.00
mean,4.58,46651.55,201.47,20.00,23533.75,11783.74
std,5.84,171177.95,646.51,40.75,114841.37,31573.78
min,1.00,5000.00,0.00,1.00,10.00,11.00
25%,1.00,7500.00,4.75,1.00,109.00,110.00
50%,1.00,15000.00,17.50,2.00,1264.00,1265.00
75%,14.00,30000.00,100.00,20.00,6182.75,6183.75
max,14.00,1500000.00,5375.00,150.00,999999.00,176471.00


In [176]:
nfl.prizePool.sum()

3732124

In [181]:
doubleups = nfl['name'].str.contains('Double Up').sum()
print ('Double ups количество в неделю: {} \nПроцент от общего числа турниров: {:.0f}%'.format(doubleups, doubleups/nfl.name.count()*100))

Double ups количество в неделю: 23 
Процент от общего числа турниров: 29%


In [175]:
nhl.describe()

,gameCount,prizePool,entryFee,maxEntriesPerUser,maxEntries,entryCount
count,19.00,19.00,19.00,19.00,19.00,19.00
mean,8.68,19127.37,199.66,24.32,2197.47,2192.89
std,1.38,29067.91,411.19,45.69,3430.90,3432.72
min,3.00,5000.00,0.50,1.00,10.00,11.00
25%,9.00,5000.00,5.00,1.00,169.00,170.00
50%,9.00,8000.00,20.00,3.00,583.00,584.00
75%,9.00,14810.00,104.50,20.00,1703.50,1704.50
max,9.00,100000.00,1500.00,150.00,11890.00,11891.00


In [180]:
doubleups = nhl['name'].str.contains('Double Up').sum()
print ('Double ups количество в неделю: {} \nПроцент от общего числа турниров: {:.0f}%'.format(doubleups, doubleups/nhl.name.count()*100))

Double ups количество в неделю: 1 
Процент от общего числа турниров: 5%


In [133]:
def get_stats(df):
    stats = pd.DataFrame(index=df.columns, columns=['na_count', 'n_unique', 'type', 'memory_usage'])
    for col in df.columns:
        stats.loc[col] = [df[col].isna().sum(), df[col].nunique(dropna=False), df[col].dtypes, df[col].memory_usage(deep=True, index=False) / 1024**2]
    stats.loc['Overall'] = [stats['na_count'].sum(), stats['n_unique'].sum(), None, df.memory_usage(deep=True).sum() / 1024**2]
    return stats

In [64]:
print (json_data[17]['entryCount'], '\n', json_normalize(json_data[17]['prizes']))

11891 
     cash  maxFinish  minFinish tierPayoutDescriptions.Cash  value
0   1000          1          1                   $1,000.00   1000
1    350          2          2                     $350.00    350
2    200          3          3                     $200.00    200
3    100          4          4                     $100.00    100
4     50          5          5                      $50.00     50
5     30          7          6                      $30.00     30
6     20         10          8                      $20.00     20
7     15         13         11                      $15.00     15
8     10         18         14                      $10.00     10
9      8         28         19                       $8.00      8
10     6         48         29                       $6.00      6
11     5         73         49                       $5.00      5
12     4        103         74                       $4.00      4
13     3        183        104                       $3.00      3
14

In [62]:
json_normalize(json_data[19]['prizes'])

,cash,maxFinish,minFinish,tierPayoutDescriptions.Cash,value
0,25000,1,1,"$25,000.00",25000
1,10000,2,2,"$10,000.00",10000
2,5000,3,3,"$5,000.00",5000
3,3500,4,4,"$3,500.00",3500
4,2500,5,5,"$2,500.00",2500
5,2000,7,6,"$2,000.00",2000
6,1500,10,8,"$1,500.00",1500
7,1000,13,11,"$1,000.00",1000
8,700,20,14,$700.00,700
9,600,29,21,$600.00,600


In [52]:
range(7, 9, 1)

range(7, 9)

In [54]:
for i in range (7, 10 ,1):
    print (i)

7
8
9
